In [1]:
# import libraries

from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, lit, count, when, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, BooleanType
from datetime import datetime
from delta.tables import DeltaTable
import os


StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 3, Finished, Available, Finished)

In [2]:
# initialize session
spark = SparkSession.builder \
    .appName("run_materialization") \
    .getOrCreate()

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 4, Finished, Available, Finished)

#### Input parameters

In [3]:
workspace = 'BUNN_Foundation_NONPROD'
lakehouse = 'silver_sapecc_lakehouse'
inputsourceschema = 'materialized_etl'
inputsource = 'orders_etl'
outputtargetschema = 'materialized_t'
outputtarget = 'orders'
update_control_table = 0
load_option = 'TR' 
run_start = datetime.now()


StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 5, Finished, Available, Finished)

In [4]:
# Begin materialization process

spark.sql(f"""
INSERT INTO utilities_lakehouse.materialization_log (schema_name, table_name, job_run_timestamp, run_id, run_step, run_timestamp, record_count, step_fail,statement_text, lakehouse_name)
VALUES ('{inputsourceschema}', '{inputsource}', '{run_start}', 10, 'begin materialization', current_timestamp(),null,null, null,'{lakehouse}')
""")

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 6, Finished, Available, Finished)

DataFrame[]

In [5]:
# insert/update materialization control table

if update_control_table == 1:
    # update materialization_control if update_control_table = 1
    query = f"""
    UPDATE materialization_control
    SET control_start_timestamp = current_timestamp()
    WHERE etl_database = '{inputsourceschema}'
    AND etl_view = '{inputsource}'
    AND lakehouse_name = '{lakehouse}'
    AND load_option = '{load_option}'
    """
    spark.sql(query)    
else:
    # insert into materialization_control if no matching record exists
    query = f"""
    INSERT INTO materialization_control (etl_database, etl_view, status_code, load_option, extract_start_timestamp, 
                            extract_end_timestamp, script_header, last_script_generate_timestamp, is_active, control_start_timestamp, lakehouse_name)
    SELECT '{inputsourceschema}', '{inputsource}', 'active', '{load_option}', '{run_start}',null, null, null, null, null, '{lakehouse}'
    WHERE NOT EXISTS (
        SELECT 1
        FROM materialization_control
        WHERE etl_database = '{inputsourceschema}'
        AND etl_view = '{inputsource}'
        AND load_option = '{load_option}'
        AND lakehouse_name = '{lakehouse}'
    )
    """
    spark.sql(query)

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 7, Finished, Available, Finished)

In [6]:
# update materialization load table execution times

spark.sql(f"""
update materialization_load
    set last_execution = current_timestamp
    WHERE 1=1
    and lakehouse_name = '{lakehouse}'
    and input_source_schema = '{inputsourceschema}'
    and input_source = '{inputsource}'
    and output_target_schema = '{outputtargetschema}'
    and output_target = '{outputtarget}'
    and load_option = '{load_option}'
""")

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 8, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint]

#### Metadata checks

In [7]:
# define input/output table path

#source_table_path  = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables/{inputsourceschema}/{inputsource}"
target_table_path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables/{outputtargetschema}/{outputtarget}"
log_table_path = f"abfss://{workspace}@onelake.dfs.fabric.microsoft.com/utilities_lakehouse.Lakehouse/Tables/materialization_log"


StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 9, Finished, Available, Finished)

In [8]:
# define log schema explicitly

log_schema = StructType([
    StructField("lakehouse_name", StringType(), True),
    StructField("schema_name", StringType(), True),
    StructField("table_name", StringType(), True),
    StructField("job_run_timestamp", TimestampType(), True),
    StructField("run_id", IntegerType(), True), 
    StructField("run_step", StringType(), True),
    StructField("run_timestamp", TimestampType(), True),
    StructField("record_count", IntegerType(), True),
    StructField("step_fail", StringType(), True),
    StructField("statement_text", StringType(), True)
])

# insert log function

def insert_log(lakehouse_name, schema_name, table_name, job_run_timestamp,
             run_id, run_step, run_timestamp, record_count, step_fail, statement_text):
    log_data = [
        Row(
            lakehouse_name=lakehouse_name,
            schema_name=schema_name,
            table_name=table_name,
            job_run_timestamp=job_run_timestamp,
            run_id=run_id,
            run_step=run_step,
            run_timestamp=run_timestamp,
            record_count = record_count,
            step_fail=step_fail,
            statement_text=statement_text
        )
    ]
    log_df = spark.createDataFrame(log_data, schema=log_schema)
    log_df.write.format("delta").mode("append").save(log_table_path)

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 10, Finished, Available, Finished)

##### Reading input data from a temporary view

In [9]:
# Input Source (Temporary View from notebook)
is_source_temp_view = False 

# actual name to use with spark.table()
source_temp_view_name_for_spark = ""

# current_source_logical_name 
current_source_logical_name = f"{inputsourceschema}.{inputsource}"

print(f"Input Source Determination for '{current_source_logical_name}'")

notebook_to_run = inputsource 

temp_view_expected_name_for_spark = inputsource
print(f"Expecting temp view from notebook. Attempting to run notebook: '{notebook_to_run}' to create temp view: '{temp_view_expected_name_for_spark}'")
try:
    exit_value = mssparkutils.notebook.run(notebook_to_run, timeout_seconds=1200, arguments={"useRootDefaultLakehouse": True})

    if spark.catalog.tableExists(temp_view_expected_name_for_spark):
        is_source_temp_view = True
        source_temp_view_name_for_spark = temp_view_expected_name_for_spark
        source_table_path = "" # Explicitly set to empty as it's not a physical path source
        print(f"SUCCESS: Ran notebook '{notebook_to_run}'. Temporary view '{source_temp_view_name_for_spark}' is available.")
        insert_log(
            lakehouse_name=lakehouse,
            schema_name=inputsourceschema,
            table_name=inputsource,
            job_run_timestamp=run_start,
            run_id=15,
            run_step=f'Success: Executed notebook {notebook_to_run} for temp view',
            run_timestamp=datetime.now(),
            record_count=None,
            step_fail=None,
            statement_text=f"Using temp view: {current_source_logical_name} (actual Spark name: {source_temp_view_name_for_spark})"
        )
    else:
        error_msg = f"Notebook '{notebook_to_run}' ran, but temp view '{temp_view_expected_name_for_spark}' was NOT created."
        print(f"ERROR: {error_msg}")
        insert_log(
            lakehouse_name=lakehouse,
            schema_name=inputsourceschema,
            table_name=inputsource,
            job_run_timestamp=run_start,
            run_id=16,
            run_step=f'Failure: Temp view not created by {notebook_to_run}',
            run_timestamp=datetime.now(),
            record_count=1,
            step_fail="True",
            statement_text=error_msg
        )
        raise Exception(error_msg)
except Exception as e_notebook_run:
    error_msg = f"Failed to find/run notebook '{notebook_to_run}', or the notebook itself failed: {str(e_notebook_run)}"
    print(f"ERROR: {error_msg}")
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,
        table_name=inputsource,
        job_run_timestamp=run_start,
        run_id=17,
        run_step=f'Failure: Executing source notebook {notebook_to_run}',
        run_timestamp=datetime.now(),
        record_count=1,
        step_fail="True",
        statement_text=error_msg
    )
    raise Exception(error_msg)

print(f"Input Source Determination Complete. Using Logical Source: '{current_source_logical_name}'")
# Modified the print output here:
if is_source_temp_view:
    print(f"Actual Spark reference for temp view: '{source_temp_view_name_for_spark}'")
else:
    # This case implies the notebook run failed to set is_source_temp_view,
      print(f"WARNING: Temp view '{temp_view_expected_name_for_spark}' was expected but 'is_source_temp_view' is False. This indicates a problem in the notebook execution flow.")

# function to read the source dataframe consistently
def get_source_dataframe():
    if is_source_temp_view:
        print(f"Reading from temporary view: {source_temp_view_name_for_spark}")
        return spark.table(source_temp_view_name_for_spark)
    else:
        # If is_source_temp_view is False here, it means the notebook run for view creation failed 
        error_msg_get_df = f"Critical Error: Expected source to be temporary view '{source_temp_view_name_for_spark}', but 'is_source_temp_view' is False. View was not properly created or identified."
        print(f"ERROR in get_source_dataframe: {error_msg_get_df}")
        raise Exception(error_msg_get_df)

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 11, Finished, Available, Finished)

Input Source Determination for 'materialized_etl.orders_etl'
Expecting temp view from notebook. Attempting to run notebook: 'orders_etl' to create temp view: 'orders_etl'


SUCCESS: Ran notebook 'orders_etl'. Temporary view 'orders_etl' is available.
Input Source Determination Complete. Using Logical Source: 'materialized_etl.orders_etl'
Actual Spark reference for temp view: 'orders_etl'


In [10]:
#1 Check if the source temporary view can be accessed

try:
    print(f"Validating determined source (Temporary View): {current_source_logical_name}")
    # The get_source_dataframe() call itself will fail if is_source_temp_view is false
    # or if the view (source_temp_view_name_for_spark) doesn't exist.
    source_df = get_source_dataframe() 
    source_df.limit(0).collect() # Spark action to trigger read and validation of the view

    # The is_source_temp_view check is now somewhat redundant here if get_source_dataframe() is robust,
    # but kept for explicit logging.
    if is_source_temp_view:
        validation_statement_text = f"Temporary View '{current_source_logical_name}' successfully validated by read attempt. (Actual Spark view: {source_temp_view_name_for_spark})"
    else:
        # This case should ideally not be reached if the previous cell's logic is sound.
        validation_statement_text = f"ERROR: Expected a temporary view, but 'is_source_temp_view' is False. Source '{current_source_logical_name}' could not be validated as a view."
        # Raise an immediate error here as it's a logic contradiction
        raise Exception("Source validation logic error: Expected temporary view, but not identified as such.")


    print(f"SUCCESS: {validation_statement_text}")
    
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,
        table_name=inputsource,
        job_run_timestamp=run_start,
        run_id=20,
        run_step='Success: Source temporary view validation', # Modified step name
        run_timestamp=datetime.now(),
        record_count=None,
        step_fail=None,
        statement_text=validation_statement_text
    )
except Exception as e:
    error_message_detail = f"Validation failed for source temporary view '{current_source_logical_name}': {str(e)}"
    print(f"ERROR: {error_message_detail}")
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,
        table_name=inputsource,
        job_run_timestamp=run_start,
        run_id=21,
        run_step='Failure: Source temporary view validation', # Modified step name
        run_timestamp=datetime.now(),
        record_count=1,
        step_fail="True",
        statement_text=error_message_detail
    )
    raise Exception(f"Source temporary view validation failed for '{current_source_logical_name}': {e}")

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 12, Finished, Available, Finished)

Validating determined source (Temporary View): materialized_etl.orders_etl
Reading from temporary view: orders_etl
SUCCESS: Temporary View 'materialized_etl.orders_etl' successfully validated by read attempt. (Actual Spark view: orders_etl)


In [11]:
#2 Check if the target table exist
try:
    if not mssparkutils.fs.exists(target_table_path):
        raise Exception(f"Target table {outputtarget} does not exist.")
      # print(f"Target table {outputtarget} exists.")
 
    # success
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,  
        table_name=inputsource,  
        job_run_timestamp=run_start,  
        run_id=30, 
        run_step='Success: Target object validation', 
        run_timestamp=datetime.now(), 
        record_count = None,
        step_fail=None, 
        statement_text=f"Check table {target_table_path}" 
    )
except Exception as e:
    # failure
    insert_log(
        lakehouse_name=lakehouse, 
        schema_name=inputsourceschema, 
        table_name=inputsource, 
        job_run_timestamp=run_start, 
        run_id=31, 
        run_step='Failure: Target object validation', 
        run_timestamp=datetime.now(), 
        record_count = 1,
        step_fail=True,  
        statement_text=f"Check table {target_table_path}" 
    )
    raise Exception(f"Target table {outputtarget} does not exist.")


StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 13, Finished, Available, Finished)

In [12]:
#3 source and traget columns check

try:
    source_df = get_source_dataframe()
    target_df = spark.read.format("delta").load(target_table_path)

    source_column_count = len(source_df.columns)
    target_column_count = len(target_df.columns)

    if not source_column_count == target_column_count:
        error_msg = f"Column count mismatch. Source ({current_source_logical_name}): {source_column_count}, Target ({outputtargetschema}.{outputtarget}): {target_column_count}"
        raise Exception(error_msg)
    
    success_msg = f"SUCCESS: Column counts match for source '{current_source_logical_name}' and target '{outputtargetschema}.{outputtarget}'."
    print(success_msg)
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,
        table_name=inputsource,
        job_run_timestamp=run_start,
        run_id=40,
        run_step='Success: Column counts match',
        run_timestamp=datetime.now(),
        record_count=None,
        step_fail=None,
        statement_text=success_msg
    )
except Exception as e:
    error_message_detail = f"Column count check failed for source '{current_source_logical_name}': {str(e)}"
    print(f"ERROR: {error_message_detail}")
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,
        table_name=inputsource,
        job_run_timestamp=run_start,
        run_id=41,
        run_step='Failure: Missing column(s) in source/target object',
        run_timestamp=datetime.now(),
        record_count=1,
        step_fail="True",
        statement_text=error_message_detail
    )
    raise Exception(f"Column count check failed: {e}")

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 14, Finished, Available, Finished)

Reading from temporary view: orders_etl
SUCCESS: Column counts match for source 'materialized_etl.orders_etl' and target 'materialized_t.orders'.


In [13]:
#4 Column name mismatch
try:
    source_columns = set(source_df.columns)
    target_columns = set(target_df.columns)

    # check for column mismatch
    column_name_mismatches = source_columns.symmetric_difference(target_columns)

    if column_name_mismatches:
        # mismatched column names list
        mismatch_details = ", ".join(column_name_mismatches)
        
        # log failure
        insert_log(
            lakehouse_name=lakehouse, 
            schema_name=inputsourceschema, 
            table_name=inputsource, 
            job_run_timestamp=run_start, 
            run_id=51, 
            run_step='Failure: Column names mismatch', 
            run_timestamp=datetime.now(), 
            record_count=len(column_name_mismatches),
            step_fail=True,  
            statement_text=f"Column name mismatches found {mismatch_details}" 
        )
        # Raise exception with the mismatch details
        raise Exception(f"Column name mismatches found {mismatch_details}")
    
    # Log success if no mismatches are found
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,  
        table_name=inputsource,  
        job_run_timestamp=run_start,  
        run_id=50, 
        run_step='Success: Column names match', 
        run_timestamp=datetime.now(), 
        record_count=None,
        step_fail=None, 
        statement_text=f"Check column names for {inputsource}" 
    )
except Exception as e:
    # catching unexpected errors
    raise Exception(f"Error during column name check: {str(e)}")

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 15, Finished, Available, Finished)

In [14]:
#5 Column datatype mismatch
try:
    data_type_mismatches = []

    for column in source_df.columns:
        if column in target_df.columns:
            # compare data types
            if str(source_df.schema[column].dataType) != str(target_df.schema[column].dataType):
                data_type_mismatches.append((column, str(source_df.schema[column].dataType), str(target_df.schema[column].dataType)))

    if data_type_mismatches:
        # mismatched columns 
        mismatch_details = "\n".join([f"Column: {col}, Source Data Type: {src_type}, Target Data Type: {tgt_type}" 
                                     for col, src_type, tgt_type in data_type_mismatches])
        
        # log failure with mismatch
        insert_log(
            lakehouse_name=lakehouse, 
            schema_name=inputsourceschema, 
            table_name=inputsource, 
            job_run_timestamp=run_start, 
            run_id=61, 
            run_step='Failure: Column data type mismatch', 
            run_timestamp=datetime.now(), 
            record_count=len(data_type_mismatches),
            step_fail=True,  
            statement_text=f"Data type mismatches found in the following columns:\n{mismatch_details}" 
        )
        raise Exception(f"Data type mismatches found in the following columns:\n{mismatch_details}")
    
    # log sucsess
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,  
        table_name=inputsource,  
        job_run_timestamp=run_start,  
        run_id=60, 
        run_step='Success: Column data types match', 
        run_timestamp=datetime.now(), 
        record_count=None,
        step_fail=None, 
        statement_text=f"Check column data types for {inputsource}" 
    )
except Exception as e:
    # raise unexpected errors
    raise Exception(f"Error during data type check {str(e)}")

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 16, Finished, Available, Finished)

#### Data loads

#### 1. Truncate and Reload

In [15]:
# 1# Truncate and Reload (TR)
if load_option == "TR":
    # Outer try-except for the whole TR process
    try:
        print(f"TR Load Option: Initiating for target {target_table_path}")
        
        # Get the source dataframe first. If this fails, we don't attempt any writes.
        try:
            source_df = get_source_dataframe() # Getting from the function
            source_record_count = source_df.count() # Get count for logging
            print(f"TR Load: Source data retrieved with {source_record_count} records.")
        except Exception as source_error:
            # Log failure related to getting source data
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema, 
                table_name=inputsource,     
                job_run_timestamp=run_start,
                run_id=190,
                run_step="Execution Error: Retrieving source data for TR",
                run_timestamp=datetime.now(),
                record_count=1,
                step_fail="True",
                statement_text=f"Error retrieving source for TR - {str(source_error)}"
            )
            print(f"FAIL (190) TR: Failed to retrieve source data. Error - {str(source_error)}")
            raise source_error # Re-raise to be caught by the outer general_error handler

        try:
            # The overwrite operation effectively.
            print(f"TR Load: Attempting to overwrite {target_table_path} with new data...")
            source_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(target_table_path)
            print(f"TR Load: Overwrite operation completed successfully for {target_table_path}.")

            # Log success for clearing of old data by overwrite
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=outputtargetschema, 
                table_name=outputtarget,
                job_run_timestamp=run_start,
                run_id=101, 
                run_step="Success: Target object cleared (by overwrite)", 
                run_timestamp=datetime.now(),
                record_count=None,
                step_fail=None,
                statement_text=f"Target {target_table_path} cleared as part of overwrite."
            )

            # Log success for to  writing of new data by overwrite
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=outputtargetschema, 
                table_name=outputtarget,
                job_run_timestamp=run_start,
                run_id=102,
                run_step="Success: New data loaded to target object (by overwrite)", 
                run_timestamp=datetime.now(),
                record_count=source_record_count, 
                step_fail=None,
                statement_text=f"New data loaded to {target_table_path} as part of overwrite."
            )

        except Exception as overwrite_error:
            error_message_detail = f"Overwrite operation failed for {target_table_path}. Error - {str(overwrite_error)}"
            print(f"FAIL (191) TR: Overwrite failed. {error_message_detail}") 
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=outputtargetschema,
                table_name=outputtarget,
                job_run_timestamp=run_start,
                run_id=191, 
                run_step="Execution Error: Overwrite target object", 
                run_timestamp=datetime.now(),
                record_count=1,
                step_fail="True",
                statement_text=error_message_detail
            )
            raise overwrite_error # Re-raise to be caught by general_error handler
    except Exception as error:
        # Raise the error to be caught by the general failure block
        raise error

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 17, Finished, Available, Finished)

TR Load Option: Initiating for target abfss://BUNN_Foundation_NONPROD@onelake.dfs.fabric.microsoft.com/silver_sapecc_lakehouse.Lakehouse/Tables/materialized_t/orders
Reading from temporary view: orders_etl
TR Load: Source data retrieved with 3934 records.
TR Load: Attempting to overwrite abfss://BUNN_Foundation_NONPROD@onelake.dfs.fabric.microsoft.com/silver_sapecc_lakehouse.Lakehouse/Tables/materialized_t/orders with new data...
TR Load: Overwrite operation completed successfully for abfss://BUNN_Foundation_NONPROD@onelake.dfs.fabric.microsoft.com/silver_sapecc_lakehouse.Lakehouse/Tables/materialized_t/orders.


#### 2. LDUI (Logical Delete Update Insert)

In [16]:
# Logical delete, update and insert
if load_option == "LDUI":
    try:
        try:
            # Query metadata table
            meta_query = f"""
            SELECT source_key, target_key
            FROM bunn_meta
            WHERE source_schema = '{inputsourceschema}'
            AND source_table = '{inputsource}'
            AND target_schema = '{outputtargetschema}'
            AND target_table = '{outputtarget}'
            """
            meta_df = spark.sql(meta_query)

            # Check PK's exist for target
            if meta_df.count() == 0:
                # Log failure for primary key check
                insert_log(
                    lakehouse_name=lakehouse,
                    schema_name=inputsourceschema,
                    table_name=inputsource,
                    job_run_timestamp=run_start,
                    run_id=291,
                    run_step="Execution Error: Primary key metadata not found",
                    run_timestamp=datetime.now(),
                    record_count=1,
                    step_fail="True",
                    statement_text=f"No primary key metadata found for target_schema={inputsourceschema}, and target_table={inputsource} in bunn_meta table."
                )
                raise ValueError(f"No primary key metadata found for target_schema={inputsourceschema}, and target_table={inputsource} in bunn_meta table.")

            # Log success for primary key check
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema,
                table_name=inputsource,
                job_run_timestamp=run_start,
                run_id=201, 
                run_step="Success: Primary key metadata found",
                run_timestamp=datetime.now(),
                record_count=meta_df.count(),
                step_fail=None,
                statement_text=f"Primary keys: {meta_df.collect()}"
            )

           # Collect source and target key columns
            primary_key_metadata = meta_df.collect()
            source_keys = [row["source_key"] for row in primary_key_metadata]
            target_keys = [row["target_key"] for row in primary_key_metadata]

            # Construct the merge condition dynamically
            merge_conditions = [
                f"target.{target_key} = source.{source_key}"
                for source_key, target_key in zip(source_keys, target_keys)
            ]
            merge_condition = " AND ".join(merge_conditions)

            # Load source and target tables
            source_df = get_source_dataframe()
            target_df = DeltaTable.forPath(spark, target_table_path)

            # Get a list of columns from source and target
            source_columns = source_df.columns
            target_columns = target_df.toDF().columns

            # Exclude audit columns
            non_key_columns = [
                col for col in source_columns
                if col not in source_keys  # Exclude audit columns
                and col not in ["action_type", "row_insert_timestamp", "row_update_timestamp"] 
            ]

            # Construct the update condition to check if any non-key column has changed
            update_conditions = [
                f"source.{col} <> target.{col}" 
                for col in non_key_columns
            ]
            update_condition = " OR ".join(update_conditions)

            set_clause = {
                col: f"source.{col}" for col in non_key_columns  
            }
            set_clause.update({
                "action_type": "'U'",  # Set action_type to 'U' for updates
                "row_update_timestamp": "current_timestamp()"  # Update row_update_timestamp
            })

            # Perform merge operation
            target_df.alias("target").merge(
                source_df.alias("source"),
                merge_condition
            ).whenMatchedUpdate(
                condition=update_condition, 
                set=set_clause  
            ).whenNotMatchedInsertAll(
            ).execute()

            print("LDUI load completed successfully")

            # Log for success
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema,
                table_name=inputsource,
                job_run_timestamp=run_start,
                run_id=202, 
                run_step="Success: LDUI load completed",
                run_timestamp=datetime.now(),
                record_count=source_df.count(),
                step_fail=None,
                statement_text=None
            )

        except Exception as merge_error:
            # Log for failure
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema,
                table_name=inputsource,
                job_run_timestamp=run_start,
                run_id=292,
                run_step="Execution Error: LDUI load failed",
                run_timestamp=datetime.now(),
                record_count=1,
                step_fail="True",
                statement_text=f"Error - {str(merge_error)}"
            )
            print(f"FAIL (292) LDUI load failed. Error - {str(merge_error)}")
            raise merge_error

    except Exception as error:
        # Raise the error for general failure
        raise error

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 18, Finished, Available, Finished)

#### 3. DLDUI (Derived Logical Delete, Update & Insert)
#### Used to refresh aggregated or synthesized objects where no action type is available from source

In [17]:
# Derived logical delete, update and insert
if load_option == "DLDUI":
    try:
        try:
            # Query metadata table
            meta_query = f"""
            SELECT source_key, target_key
            FROM bunn_meta
            WHERE source_schema = '{inputsourceschema}'
            AND source_table = '{inputsource}'
            AND target_schema = '{outputtargetschema}'
            AND target_table = '{outputtarget}'
            """
            meta_df = spark.sql(meta_query)

            # Check PK's exist for target
            if meta_df.count() == 0:
                # Log failure for primary key check
                insert_log(
                    lakehouse_name=lakehouse,
                    schema_name=inputsourceschema,
                    table_name=inputsource,
                    job_run_timestamp=run_start,
                    run_id=391,
                    run_step="Execution Error: Primary key metadata not found",
                    run_timestamp=datetime.now(),
                    record_count=1,
                    step_fail="True",
                    statement_text=f"No primary key metadata found for target_schema={inputsourceschema}, and target_table={inputsource} in bunn_meta table."
                )
                raise ValueError(f"No primary key metadata found for target_schema={inputsourceschema}, and target_table={inputsource} in bunn_meta table.")

            # Log success for primary key check
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema,
                table_name=inputsource,
                job_run_timestamp=run_start,
                run_id=301, 
                run_step="Success: Primary key metadata found",
                run_timestamp=datetime.now(),
                record_count=meta_df.count(),
                step_fail=None,
                statement_text=f"Primary keys: {meta_df.collect()}"
            )

           # Collect source and target key columns
            primary_key_metadata = meta_df.collect()
            source_keys = [row["source_key"] for row in primary_key_metadata]
            target_keys = [row["target_key"] for row in primary_key_metadata]

            # Construct the merge condition dynamically
            merge_conditions = [
                f"target.{target_key} = source.{source_key}"
                for source_key, target_key in zip(source_keys, target_keys)
            ]
            merge_condition = " AND ".join(merge_conditions)

            # Load source and target tables
            source_df = get_source_dataframe()
            target_df = DeltaTable.forPath(spark, target_table_path)

            # Get a list of columns from source and target
            source_columns = source_df.columns
            target_columns = target_df.toDF().columns

            # Exclude audit columns
            non_key_columns = [
                col for col in source_columns
                if col not in source_keys  # Exclude primary key columns
                and col not in ["action_type", "row_insert_timestamp", "row_update_timestamp"] 
            ]

            # Construct the update condition to check if any non-key column has changed
            update_conditions = [
                f"source.{col} <> target.{col}" 
                for col in non_key_columns
            ]
            update_condition = " OR ".join(update_conditions)

            set_clause = {
                col: f"source.{col}" for col in non_key_columns  
            }
            set_clause.update({
                "action_type": "'U'",  # Set action_type to 'U' for updates
                "row_update_timestamp": "current_timestamp()"  # Update row_update_timestamp
            })

            # Perform merge operation
            target_df.alias("target").merge(
                source_df.alias("source"),
                merge_condition
            ).whenMatchedUpdate(
                condition=update_condition, 
                set=set_clause  
            ).whenNotMatchedInsertAll(
            ).whenNotMatchedBySourceUpdate(
                condition="target.action_type != 'D'",  # get records with action_type <> 'D'
                set={
                    "action_type": "'D'",  # Set action_type to 'D' for deleted records
                    "row_update_timestamp": "current_timestamp()"  # Update row_update_timestamp
                }
            ).execute()

            print("DLDUI load completed successfully")

            # Log for success
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema,
                table_name=inputsource,
                job_run_timestamp=run_start,
                run_id=302, 
                run_step="Success: DLDUI load completed",
                run_timestamp=datetime.now(),
                record_count=source_df.count(),
                step_fail=None,
                statement_text=None
            )

        except Exception as merge_error:
            # Log for failure
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema,
                table_name=inputsource,
                job_run_timestamp=run_start,
                run_id=392,
                run_step="Execution Error: DLDUI load failed",
                run_timestamp=datetime.now(),
                record_count=1,
                step_fail="True",
                statement_text=f"Error - {str(merge_error)}"
            )
            print(f"FAIL (392) DLDUI load failed. Error - {str(merge_error)}")
            raise merge_error

    except Exception as error:
        # Raise the error for general failure
        raise error

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 19, Finished, Available, Finished)

### 4. CLDUI (Controlled Logical Delete Update & Insert)
#### Performs targeted updates controlled by timestamp from last successful execution



In [18]:
# Controlled Logical delete, update and insert

if load_option == "CLDUI":
    try:
        try:
            # Query metadata table
            meta_query = f"""
            SELECT source_key, target_key
            FROM bunn_meta
            WHERE source_schema = '{inputsourceschema}'
            AND source_table = '{inputsource}'
            AND target_schema = '{outputtargetschema}'
            AND target_table = '{outputtarget}'
            """
            meta_df = spark.sql(meta_query)

            # Check PK's exist for target
            if meta_df.count() == 0:
                # Log failure for primary key check
                insert_log(
                    lakehouse_name=lakehouse,
                    schema_name=inputsourceschema,
                    table_name=inputsource,
                    job_run_timestamp=run_start,
                    run_id=491,
                    run_step="Execution Error: Primary key metadata not found",
                    run_timestamp=datetime.now(),
                    record_count=1,
                    step_fail="True",
                    statement_text=f"No primary key metadata found for target_schema={inputsourceschema}, and target_table={inputsource} in bunn_meta table."
                )
                raise ValueError(f"No primary key metadata found for target_schema={inputsourceschema}, and target_table={inputsource} in bunn_meta table.")

            # Log success for primary key check
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema,
                table_name=inputsource,
                job_run_timestamp=run_start,
                run_id=401, 
                run_step="Success: Primary key metadata found",
                run_timestamp=datetime.now(),
                record_count=meta_df.count(),
                step_fail=None,
                statement_text=f"Primary keys: {meta_df.collect()}"
            )

           # Collect source and target key columns
            primary_key_metadata = meta_df.collect()
            source_keys = [row["source_key"] for row in primary_key_metadata]
            target_keys = [row["target_key"] for row in primary_key_metadata]

            # Construct the merge condition dynamically
            merge_conditions = [
                f"target.{target_key} = source.{source_key}"
                for source_key, target_key in zip(source_keys, target_keys)
            ]
            merge_condition = " AND ".join(merge_conditions)

            # Load source and target tables
            source_df = get_source_dataframe()
            target_df = DeltaTable.forPath(spark, target_table_path)

            # Get a list of columns from source and target
            source_columns = source_df.columns
            target_columns = target_df.toDF().columns

            # Exclude audit columns
            non_key_columns = [
                col for col in source_columns
                if col not in source_keys  # Exclude audit key columns
                and col not in ["action_type", "row_insert_timestamp", "row_update_timestamp"] 
            ]

            # Construct the update condition to check if any non-key column has changed
            update_conditions = [
                f"source.{col} <> target.{col}" 
                for col in non_key_columns
            ]
            update_condition = " OR ".join(update_conditions)

            set_clause = {
                col: f"source.{col}" for col in non_key_columns  
            }
            set_clause.update({
                "action_type": "'U'",  # Set action_type to 'U' for updates
                "row_update_timestamp": "current_timestamp()"  # Update row_update_timestamp
            })

            # Fetch control_start_timestamp from materialization_control_table
            control_table_query = f"""
                SELECT control_start_timestamp
                FROM materialization_control
                WHERE etl_database = '{inputsourceschema}'
                AND etl_view = '{inputsource}'
                and load_option ='{load_option}'
                """

            control_df = spark.sql(control_table_query)  
            control_start_timestamp = "1900-01-01 00:00:00"

            # Check if control_df has rows
            if control_df.count() > 0:
                # Get the first row
                row = control_df.collect()[0]
                # Check if the value is not None before using it
                if row["control_start_timestamp"] is not None:
                    control_start_timestamp = row["control_start_timestamp"]

            # Pull incremental data from the source table
            source_df = get_source_dataframe() \
            .filter(f"row_update_timestamp >= '{control_start_timestamp}'")

            # Perform merge operation
            target_df.alias("target").merge(
                source_df.alias("source"),
                merge_condition
            ).whenMatchedUpdate(
                condition=update_condition, 
                set=set_clause  
            ).whenNotMatchedInsertAll(
            ).execute()

            print("CLDUI load completed successfully")

            # Log for success
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema,
                table_name=inputsource,
                job_run_timestamp=run_start,
                run_id=402, 
                run_step="Success: CLDUI load completed",
                run_timestamp=datetime.now(),
                record_count=source_df.count(),
                step_fail=None,
                statement_text=None
            )

        except Exception as merge_error:
            # Log for failure
            insert_log(
                lakehouse_name=lakehouse,
                schema_name=inputsourceschema,
                table_name=inputsource,
                job_run_timestamp=run_start,
                run_id=492,
                run_step="Execution Error: CLDUI load failed",
                run_timestamp=datetime.now(),
                record_count=1,
                step_fail="True",
                statement_text=f"Error - {str(merge_error)}"
            )
            print(f"FAIL (492) CLDUI load failed. Error - {str(merge_error)}")
            raise merge_error

    except Exception as error:
        # Raise the error for general failure
        raise error

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 20, Finished, Available, Finished)

In [ ]:
control_table_query = f"""
    SELECT control_start_timestamp
    FROM materialization_control
    WHERE etl_database = '{inputsourceschema}'
    AND etl_view = '{inputsource}'
    and load_option ='{load_option}'
    """
control_df = spark.sql(control_table_query)  
control_start_timestamp = "1900-01-01 00:00:00"

# Check if control_df has rows
if control_df.count() > 0:
    # Get the first row
    row = control_df.collect()[0]
    # Check if the value is not None before using it
    if row["control_start_timestamp"] is not None:
        control_start_timestamp = row["control_start_timestamp"]


# Pull incremental data from the source table
source_df = spark.read.format("delta").load(source_table_path) \
   .filter(f"row_update_timestamp >= '{control_start_timestamp}'")


In [43]:
# Define the query
control_table_query = f"""
    SELECT control_start_timestamp
    FROM materialization_control
    WHERE etl_database = '{inputsourceschema}'
    AND etl_view = '{inputsource}'
    AND load_option = '{load_option}'
"""

# Execute the query
control_df = spark.sql(control_table_query)
control_start_timestamp = "1900-01-01 00:00:00"


# Check if control_df has rows
if control_df.count() > 0:
    # Get the first row
    row = control_df.collect()[0]
    # Check if the value is not None before using it
    if row["control_start_timestamp"] is not None:
        control_start_timestamp = row["control_start_timestamp"]

# Print the control_start_timestamp
print("Control Start Timestamp:", control_start_timestamp)

StatementMeta(, 6f1f94b2-98cf-4d7d-9b79-b816427ec58c, 45, Finished, Available, Finished)

Control Start Timestamp: 1900-01-01 00:00:00


In [19]:
# Check if source_df has any records
if source_df.count() > 0:
    # Get max timestamp from source table
    max_row_update_timestamp = source_df.selectExpr("max(row_update_timestamp)").collect()[0][0]

    # Update the control_start_timestamp in materialization_control table
    control_update_query = f"""
        MERGE INTO materialization_control AS target
        USING (
            SELECT '{inputsourceschema}' AS etl_database,
                   '{inputsource}' AS etl_view,
                   '{load_option}' AS load_option,
                   to_timestamp('{max_row_update_timestamp}') AS control_start_timestamp
        ) AS source
        ON target.etl_database = source.etl_database
           AND target.etl_view = source.etl_view
           AND target.load_option = source.load_option
        WHEN MATCHED THEN
            UPDATE SET target.control_start_timestamp = source.control_start_timestamp
        WHEN NOT MATCHED THEN
            INSERT (etl_database, etl_view, load_option, control_start_timestamp)
            VALUES (source.etl_database, source.etl_view, source.load_option, source.control_start_timestamp)
    """
    print(spark.sql(control_update_query))
else:
    print("No records to process. Skipping update of control_start_timestamp.")

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 21, Finished, Available, Finished)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]


#### Exception for unsupported load option

In [20]:
# Unsupported Load Option
if load_option not in ["TR","LDUI","DLDUI","CLDUI"]:
    try:
        raise ValueError(f" Unsupported load_option - {load_option}")
    except Exception as e:
        # Log failure for unsupported load_option
        insert_log(
            lakehouse_name=lakehouse,
            schema_name=inputsourceschema,
            table_name=inputsource,
            job_run_timestamp=run_start,
            run_id=999,  
            run_step="Execution Error: Unsupported load_option",
            run_timestamp=datetime.now(),
            record_count=1,  # No change for failure
            step_fail="True",
            statement_text=f"Error - {str(e)}"
        )
        print(f"FAIL (999) Unsupported load_option. Error - {str(e)}")
        raise e

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 22, Finished, Available, Finished)

### Load complete

In [21]:
# Load complete check
if load_option == "TR":
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,
        table_name=inputsource,
        job_run_timestamp=run_start,
        run_id=200, 
        run_step="Load complete",
        run_timestamp=datetime.now(),
        record_count=None,
        step_fail=None,
        statement_text=None
    )
elif load_option == "LDUI":
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,
        table_name=inputsource,
        job_run_timestamp=run_start,
        run_id=300, 
        run_step="Load complete",
        run_timestamp=datetime.now(),
        record_count=None,
        step_fail=None,
        statement_text=None
    ) 
elif load_option == "DLDUI":
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,
        table_name=inputsource,
        job_run_timestamp=run_start,
        run_id=400, 
        run_step="Load complete",
        run_timestamp=datetime.now(),
        record_count=None,
        step_fail=None,
        statement_text=None
    )
elif load_option == "CLDUI":
    insert_log(
        lakehouse_name=lakehouse,
        schema_name=inputsourceschema,
        table_name=inputsource,
        job_run_timestamp=run_start,
        run_id=500, 
        run_step="Load complete",
        run_timestamp=datetime.now(),
        record_count=None,
        step_fail=None,
        statement_text=None
    )
    # Update materialization_control table
    spark.sql(f"""
        UPDATE materialization_control
        SET 
            extract_start_timestamp = '{run_start}',
            extract_end_timestamp = '{datetime.now()}'
        WHERE 1 = 1
        AND etl_database = '{inputsourceschema}'
        AND etl_view = '{inputsource}'
        AND load_option = '{load_option}'
        AND lakehouse_name = '{lakehouse}'
    """)

    # update materialization_load table
    spark.sql(f"""
        UPDATE materialization_load
        SET
            last_successful_execution = '{datetime.now()}'
        WHERE 1 = 1
        AND lakehouse_name = '{lakehouse}'
        AND input_source_schema = '{inputsourceschema}'
        AND input_source = '{inputsource}'
        AND output_target_schema = '{outputtargetschema}'
        AND output_target = '{outputtarget}'
        AND load_option = '{load_option}'
    """)

    print(f"Load completed successfully for load_option: {load_option}.")

StatementMeta(, 055e122a-0d87-4fa1-ad4a-f8f8239263d4, 23, Finished, Available, Finished)